In [40]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Hotel").getOrCreate()

file_path = "/content/drive/MyDrive/Colab Notebooks/Hotel.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.createOrReplaceTempView("logs_hotel")

df.printSchema()

df.show(5)


root
 |-- ID: string (nullable = true)
 |-- n_adults: integer (nullable = true)
 |-- n_children: integer (nullable = true)
 |-- weekend_nights: integer (nullable = true)
 |-- week_nights: integer (nullable = true)
 |-- meal_plan: string (nullable = true)
 |-- car_parking_space: integer (nullable = true)
 |-- room_type: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- avg_room_price: double (nullable = true)
 |-- special_requests: integer (nullable = true)
 |-- status: string (nullable = true)

+--------+--------+----------+--------------+-----------+------------+-----------------+-----------+---------+----+-----+----+--------------+--------------

In [41]:
calendar_df = spark.sql("""
    SELECT explode(sequence(
        to_date('2017-01-01'),
        to_date('2018-12-31'),
        interval 1 day
    )) as calendar_dt
""")

calendar_df.createOrReplaceTempView("calendar")

calendar_df.show(5)

+-----------+
|calendar_dt|
+-----------+
| 2017-01-01|
| 2017-01-02|
| 2017-01-03|
| 2017-01-04|
| 2017-01-05|
+-----------+
only showing top 5 rows



In [44]:
step1 = """
SELECT
    year,
    month,
    ROUND(AVG(weekend_nights + week_nights), 2) as avg_nights
FROM logs_hotel
WHERE status = 'Not_Canceled'
GROUP BY year, month
ORDER BY year, month
"""

result1 = spark.sql(step1)
result1.show()

+----+-----+----------+
|year|month|avg_nights|
+----+-----+----------+
|2017|    7|      3.02|
|2017|    8|      2.72|
|2017|    9|      2.66|
|2017|   10|       2.7|
|2017|   11|      2.72|
|2017|   12|      3.04|
|2018|    1|      2.74|
|2018|    2|      2.69|
|2018|    3|      3.04|
|2018|    4|      2.92|
|2018|    5|      2.81|
|2018|    6|       2.6|
|2018|    7|      3.19|
|2018|    8|      3.15|
|2018|    9|      2.79|
|2018|   10|      2.89|
|2018|   11|      2.98|
|2018|   12|      3.25|
+----+-----+----------+



In [43]:
step2 = """
WITH month_stats AS (
    SELECT
        month,
        COUNT(*) as total_bookings,
        SUM(CASE WHEN status = 'Canceled' THEN 1 ELSE 0 END) as canceled_count
    FROM logs_hotel
    WHERE year = 2018
    GROUP BY month
)
SELECT
    month,
    total_bookings,
    canceled_count,
    ROUND((canceled_count * 100.0 / total_bookings), 2) as cancellation_rate_percent
FROM month_stats
ORDER BY cancellation_rate_percent DESC
LIMIT 3
"""

result2 = spark.sql(step2)
result2.show()

+-----+--------------+--------------+-------------------------+
|month|total_bookings|canceled_count|cancellation_rate_percent|
+-----+--------------+--------------+-------------------------+
|    8|          2799|          1303|                    46.55|
|   10|          3404|          1578|                    46.36|
|    9|          2962|          1356|                    45.78|
+-----+--------------+--------------+-------------------------+



In [45]:
step3 = """
SELECT
    year,
    month,
    ROUND(AVG(lead_time), 2) as avg_lead_time_days,
    COUNT(*) as confirmed_bookings
FROM logs_hotel
WHERE status = 'Not_Canceled'
GROUP BY year, month
ORDER BY year, month
"""

result3 = spark.sql(step3)
result3.show()

+----+-----+------------------+------------------+
|year|month|avg_lead_time_days|confirmed_bookings|
+----+-----+------------------+------------------+
|2017|    7|            130.73|               120|
|2017|    8|             35.08|               829|
|2017|    9|             51.72|              1467|
|2017|   10|             55.89|              1611|
|2017|   11|             33.28|               620|
|2017|   12|             46.75|               906|
|2018|    1|             34.87|               990|
|2018|    2|             30.53|              1274|
|2018|    3|             43.19|              1658|
|2018|    4|             62.49|              1741|
|2018|    5|             60.99|              1650|
|2018|    6|             70.64|              1912|
|2018|    7|             86.88|              1486|
|2018|    8|             83.09|              1496|
|2018|    9|             63.32|              1606|
|2018|   10|             73.24|              1826|
|2018|   11|             44.25|

In [47]:
step4 = """
SELECT *
FROM (
    SELECT
        year,
        month,
        market_segment,
        ROUND(AVG(avg_room_price * (weekend_nights + week_nights)), 2) as avg_revenue
    FROM logs_hotel
    WHERE status = 'Not_Canceled'
    GROUP BY year, month, market_segment
)
PIVOT (
    ROUND(AVG(avg_revenue), 2)
    FOR market_segment IN ('Online', 'Offline', 'Corporate', 'Aviation', 'Complementary')
)
ORDER BY year, month
"""

result4_pivot = spark.sql(step4)
result4_pivot.show()

+----+-----+------+-------+---------+--------+-------------+
|year|month|Online|Offline|Corporate|Aviation|Complementary|
+----+-----+------+-------+---------+--------+-------------+
|2017|    7|290.56| 228.95|   113.75|    NULL|         22.4|
|2017|    8|284.21| 235.54|   156.42|    NULL|         0.32|
|2017|    9|348.55| 236.65|   177.83|    NULL|        16.89|
|2017|   10|311.47| 223.24|   180.26|    NULL|         1.09|
|2017|   11|240.52| 198.36|   102.97|    NULL|        14.81|
|2017|   12|258.93| 253.86|   141.11|    NULL|         0.25|
|2018|    1|236.09| 210.51|   113.03|    NULL|         2.27|
|2018|    2|238.07| 251.85|   115.06|   352.0|         1.39|
|2018|    3|301.71| 233.39|   142.39|  118.33|        38.17|
|2018|    4|320.08| 236.44|   108.42|  321.81|          0.0|
|2018|    5|352.34| 274.55|    229.5|   262.5|          0.0|
|2018|    6|335.03| 251.98|   148.13|   247.0|          0.0|
|2018|    7|390.05| 310.36|   146.99|    79.0|         5.38|
|2018|    8|391.22| 318.

In [50]:
step5 = """
WITH all_money AS (
    SELECT SUM(avg_room_price * (weekend_nights + week_nights)) as all_total
    FROM logs_hotel
    WHERE status = 'Not_Canceled' AND repeated_guest = 1
)
SELECT
    ID as guest_id,
    ROUND(SUM(avg_room_price * (weekend_nights + week_nights)), 2) as total_revenue,
    ROUND(SUM(avg_room_price * (weekend_nights + week_nights)) * 100.0 /
          (SELECT all_total FROM all_money), 2) as revenue_percent
FROM logs_hotel
WHERE status = 'Not_Canceled' AND repeated_guest = 1
GROUP BY ID
ORDER BY total_revenue DESC
LIMIT 5
"""

result5 = spark.sql(step5)
result5.show()

+--------+-------------+---------------+
|guest_id|total_revenue|revenue_percent|
+--------+-------------+---------------+
|INN19235|       1754.4|           1.55|
|INN05222|        690.0|           0.61|
|INN14189|        665.0|           0.59|
|INN09923|        660.0|           0.58|
|INN25479|        650.0|           0.57|
+--------+-------------+---------------+

